### Import Libraries and Load Data

In [32]:
#basic imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings

#metrics, model selection tools, scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, train_test_split
from scipy.stats import uniform

#models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 

#natural language processing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
import re

#other
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
sns.set_style("darkgrid")
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_variants_df = pd.read_csv("msk-redefining-cancer-treatment/training_variants", index_col = 0)
test_variants_df = pd.read_csv("msk-redefining-cancer-treatment/test_variants", header = None, names = train_variants_df.columns, index_col = 0)
train_text_df = pd.read_csv("msk-redefining-cancer-treatment/training_text", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"], index_col = 0)
test_text_df = pd.read_csv("msk-redefining-cancer-treatment/test_text", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"], index_col = 0)
train_variants_df = train_variants_df.merge(train_text_df, left_index = True, right_index = True)
test_variants_df = test_variants_df.merge(test_text_df, left_index = True, right_index = True)
#5 observations, all different genes, missing text information
train_variants_df.dropna(inplace=True)

### Baseline Models
We are creating as simple of a model possible to see what we can predict just from knowing hte Gene and the Variation. We will not use 'Text' nor do anything with the 'Variation' column, as this would require deliberate and consequential feature engineering choices.

We will use this as a benchmark to evaluate our models later. As baseline model classifiers, we will use logisitic regression, as it is well-suited to multiclass classification, and random forest, as it is a versatile and powerful ensemble method.

In [3]:
train_variants_df = train_variants_df.dropna() # only a handful, so ok to drop
baseline_y = train_variants_df.Class
baseline_X = train_variants_df.drop(columns = ['Text', 'Class'])
baseline_X = pd.get_dummies(baseline_X)
base_X_train, base_X_test, base_y_train, base_y_test = train_test_split(baseline_X, baseline_y, \
                                                                        test_size = .2, shuffle = True,\
                                                                        random_state = 42)

In [4]:
logisticRegr = LogisticRegression()
model_l = logisticRegr.fit(base_X_train, base_y_train)
base_l_score = model_l.score(base_X_test, base_y_test)
base_l_score 

0.5662650602409639

In [5]:
rf = RandomForestClassifier(n_estimators= 50)
model_rf = rf.fit(base_X_train, base_y_train)
base_rf_score = model_rf.score(base_X_test, base_y_test)
base_rf_score

0.5737951807228916

Our two classifiers' models hover around an accuracy of 57%.

This already is a substantial improvement over random (11.11%) 
or than if we chose the largest class (28%). 

### Loading Background Functions

In [6]:
acidic_polar = {'D': 'Aspartic Acid', 'E': 'Glutamic Acid'}
basic_polar = {'R': 'Arginine', 'H': 'Histidine', 'K': 'Lysine'}
nonpolar = {'A': 'Alanine', 'C': 'Cysteine', 'G': 'Glycine', 'I': 'Isoleucine', 'L': 'Leucine', 'M': 'Methionine',\
           'F': 'Phenylalanine', 'P': 'Proline', 'W': 'Tryptophan', 'V': 'Valine'}
polar = {'N': 'Asparagine', 'Q': 'Glutamine', 'S': 'Serine', 'T': 'Threonine', 'Y': 'Tyrosine'}

amino_acids = [acidic_polar, basic_polar, nonpolar, polar]

In [7]:
def get_mutation_type(variation):
    if 'Truncating Mutations' in variation or 'trunc' in variation:
        mutation = 'Nonsense'
    elif 'delins' in variation.lower():
        mutation = 'Deletion/Insertion'
    elif 'ins' in variation.lower():
        mutation = 'Insertion'
    elif 'fs' in variation.lower():
        mutation = 'Frameshift'
    elif 'del' in variation.lower():
        mutation = 'Deletion'
    elif 'dup' in variation.lower():
        mutation = 'Duplication'
    elif 'splice' in variation.lower():
        mutation = 'Splice'
    elif (variation == 'Deletion') or (variation == 'Amplification') or (variation == 'Fusions') or (variation == 'Overexpression'):
        mutation = variation
    elif '*' in variation.lower():
            mutation = 'Nonsense'
    elif len(variation) <= 6:
        mutation = 'Non-Conservative Missense'
        #mutation = variation[0] #+ variation[-1]
        for amino_acid in amino_acids:
            if (variation[0] in amino_acid) and (variation[-1] in amino_acid):
                mutation = 'Conservative Missense'
    elif 'Fusion' in variation:
        mutation = 'Fusions'
    elif 'prom' in variation.lower():
        mutation = 'Promoter'
    else:
        mutation = 'Other'
    return mutation

In [8]:
def lemmatize_tokenizer(str_input):
    mod = re.sub("\d", "", str_input)
    words = re.sub("\W", " ", mod).lower().split()
    words = [WordNetLemmatizer().lemmatize(word) for word in words]
    return words

In [ ]:
def edit_texts(text):
    lemmatizer = WordNetLemmatizer()
    resultwords  = [word for word in re.split("\W+", text) \
                    if lemmatizer.lemmatize(word.lower()) in top_words]
    result = ' '.join(resultwords)
    return result

### Feature Engineering and Final Cleaning of Data Sets

In [9]:
train_variants_df.head()

,Gene,Variation,Class,Text
ID,,,,
0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [10]:
text_exploration = train_variants_df[['Class','Text']].groupby(['Class'])

In [11]:
counter = CountVectorizer(stop_words = 'english', tokenizer = lemmatize_tokenizer, encoding = 'latin-1',\
                         min_df = .25, max_features = 10000, ngram_range = (1,1))

In [12]:
top_words_dfs = []
top_words = set()
for name, group in text_exploration:
    features = counter.fit_transform(group.Text).toarray()
    feature_names = counter.get_feature_names()
    top_words = top_words.union(set(feature_names))
    top_words_dfs.append(pd.DataFrame(features, columns = feature_names))

In [13]:
len(top_words)

2918

In [15]:
train_variants_df.Text = train_variants_df.Text.apply(edit_texts)

In [16]:
train_variants_df.head()

,Gene,Variation,Class,Text
ID,,,,
0,FAM58A,Truncating Mutations,1,Cyclin dependent kinases regulate variety fund...
1,CBL,W802*,2,Abstract Background Non small cell lung cancer...
2,CBL,Q249E,2,Abstract Background Non small cell lung cancer...
3,CBL,N454D,3,Recent evidence has demonstrated acquired nove...
4,CBL,L399V,4,Oncogenic mutations B lineage lymphoma Cbl gen...


In [17]:
train_variants_df['Label'] = 'Train'
test_variants_df['Label'] = 'Test'
concat_df = pd.concat([train_variants_df.dropna(), test_variants_df])

In [18]:
concat_df['Mutation Type'] = concat_df.Variation.apply(get_mutation_type)
concat_df.reset_index(inplace = True, drop = True)
concat_df = pd.get_dummies(concat_df, columns = ['Gene', 'Mutation Type'])

In [ ]:
final_counter =  CountVectorizer(stop_words = 'english',\
                         tokenizer = lemmatize_tokenizer, encoding = 'latin-1',\
                         max_df = .25, max_features = 20000, ngram_range = (1,4))

In [19]:
features = final_counter.fit_transform(concat_df['Text']).toarray()
cleaned_df = pd.DataFrame(features, columns = final_counter.get_feature_names())

In [20]:
concat_df = cleaned_df.merge(concat_df, left_index = True, right_index = True)
concat_df.head()

,_egfr_,a_yinsfqea,aa,aa aa,aacr,aacrjournals,aacrjournals org,aacrjournals org american,aacrjournals org american association,aacrjournals org cancer,...,Mutation Type_Deletion/Insertion,Mutation Type_Duplication,Mutation Type_Frameshift,Mutation Type_Fusions,Mutation Type_Insertion,Mutation Type_Non-Conservative Missense,Mutation Type_Nonsense,Mutation Type_Overexpression,Mutation Type_Promoter,Mutation Type_Splice
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Prepare Training and Testing Sets for Modeling

In [24]:
train_variants_df = concat_df[concat_df['Label'] == 'Train']
test_variants_df = concat_df[concat_df['Label'] == 'Test']

In [25]:
y_train = train_variants_df.Class
X_train = train_variants_df.drop(columns = ['Variation', 'Class', 'Text', 'Label'])

y_test = test_variants_df.Class
X_test = test_variants_df.drop(columns = ['Variation', 'Class', 'Text', 'Label'])

In [27]:
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [28]:
print(f'X_train shape: {X_train.shape} \nX_test shape: {X_test.shape}')

X_train shape: (3299, 20280) 
X_test shape: (365, 20280)


## Models

In [29]:
#cross validation using stratified Kfolds
cv = StratifiedKFold(n_splits=10, random_state= 42, shuffle = True)

# set up dictionary to capture results
results = {}
results['Baseline Logistic Regression'] = [base_l_score, np.nan]
results['Baseline Random Forest'] = [base_rf_score, np.nan]

#### Logistic Regression

In [33]:
penalty = ['l1', 'l2']
C = uniform(loc=0, scale=4)

hyperparameters = dict(C=C, penalty=penalty)

In [34]:
logisticRegr = LogisticRegression()
log_random = RandomizedSearchCV(logisticRegr, hyperparameters, cv=cv, random_state = 1, n_iter = 20)
best_logistic_model = log_random.fit(X_train, y_train)

i = np.where(best_logistic_model.cv_results_['rank_test_score'] == 1)[0][0]
mean = best_logistic_model.cv_results_['mean_test_score'][i]
std = best_logistic_model.cv_results_['std_test_score'][i]
results['Logistic Regression'] = [mean, std]

In [42]:
best_logistic_model.best_params_
# {'C': 1.2530940677291005, 'penalty': 'l1'}

{'C': 1.2530940677291005, 'penalty': 'l1'}

#### K nearest neighbors

In [35]:
n_neighbors = list(range(1,30))
leaf_size = list(range(1,50))

hyperparameters = dict(n_neighbors = n_neighbors, leaf_size = leaf_size)

In [36]:
neigh = KNeighborsClassifier()
knn_random = RandomizedSearchCV(neigh, hyperparameters, cv = cv, random_state = 1, n_iter = 20)
best_knn_model = knn_random.fit(X_train, y_train)

i = np.where(best_knn_model.cv_results_['rank_test_score'] == 1)[0][0]
mean = best_knn_model.cv_results_['mean_test_score'][i]
std = best_knn_model.cv_results_['std_test_score'][i]
results['KNN'] = [mean, std]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/sebastianmonzon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-3ef3fe545752>", line 3, in <module>
    best_knn_model = knn_random.fit(X_train, y_train)
  File "/Users/sebastianmonzon/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py", line 688, in fit
    self._run_search(evaluate_candidates)
  File "/Users/sebastianmonzon/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py", line 1469, in _run_search
    random_state=self.random_state))
  File "/Users/sebastianmonzon/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py", line 667, in evaluate_candidates
    cv.split(X, y, groups)))
  File "/Users/sebastianmonzon/anaconda3/lib/python3.6/site-packages/joblib/parallel.py", line 1006, in __call__
    while self.dispatch_one_batch(iterator):
  File "

KeyboardInterrupt: 

#### Multinomial Naïve Bayes

In [43]:
hyperparameters = {'alpha': [0, .5, 1, 1.5, 2]}

In [44]:
mnb = MultinomialNB()
mnb_gridsearch = GridSearchCV(mnb, hyperparameters, cv=cv)
best_mnb_model = mnb_gridsearch.fit(X_train, y_train)

i = np.where(best_mnb_model.cv_results_['rank_test_score'] == 1)[0][0]
mean = best_mnb_model.cv_results_['mean_test_score'][i]
std = best_mnb_model.cv_results_['std_test_score'][i]
results['MNB'] = [mean, std]

#### Random Forest

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
criteria = ['entropy', 'gini']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

hyperparameters = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'criterion': criteria,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(rf, hyperparameters, cv = cv, random_state = 1, n_iter = 50)
best_rf_model = rf_random.fit(X_train, y_train)

i = np.where(best_rf_model.cv_results_['rank_test_score'] == 1)[0][0]
mean = best_rf_model.cv_results_['mean_test_score'][i]
std = best_rf_model.cv_results_['std_test_score'][i]
results['Random Forest'] = [mean, std]

#### SVM

In [ ]:
g_range = np.random.uniform(0.0, 0.3, 5).astype(float)
C_range = np.random.normal(1, 0.1, 5).astype(float)
 
C_range[C_range < 0] = 0.0001
 
hyperparameters = {'gamma': list(g_range), 
                    'C': list(C_range)}

In [ ]:
lin_svm = svm.LinearSVC()
randomSVM = RandomizedSearchCV(lin_svm(kernel='rbf',), hyperparameters, n_iter = 20)
best_SVM_model = randomSVM.fit(XTrain, yTrain)

i = np.where(best_SVM_model.cv_results_['rank_test_score'] == 1)[0][0]
mean = best_SVM_model.cv_results_['mean_test_score'][i]
std = best_SVM_model.cv_results_['std_test_score'][i]
results['Linear SVM'] = [mean, std]

### Comparing Results and Predictions on Test Data

In [37]:
results

{'Baseline Logistic Regression': [0.5662650602409639, nan],
 'Baseline Random Forest': [0.5737951807228916, nan],
 'Logistic Regression': [0.6608063049408912, 0.025842013597734977]}

In [39]:
results_df = pd.DataFrame.from_dict(results, orient = 'index', columns = ['Accuracy', 'Standard_Deviation'])

In [40]:
results_df.sort_values(by = ['Accuracy'], ascending = False)

,Accuracy,Standard_Deviation
Logistic Regression,0.660806,0.025842
Baseline Random Forest,0.573795,NaN
Baseline Logistic Regression,0.566265,NaN


In [ ]:
chosen_model = best_rf_model

In [ ]:
confusion= pd.DataFrame(confusion_matrix(y_test, chosen_model.fit(X_train, y_train).predict(X_test)))
confusion.columns = ['pred class ' + str(x) for x in list(range(1,10))]
confusion.index = ['true class ' + str(x) for x in list(range(1,10))]
confusion